### Install FireDrone API Cient SDK

In [1]:
!pip install --index-url https://test.pypi.org/simple/ fire-drone-sdk -U

Looking in indexes: https://test.pypi.org/simple/
Requirement already up-to-date: fire-drone-sdk in c:\users\john light\appdata\local\programs\python\python36\lib\site-packages (0.2.25)


### Install dependencies

In [2]:
import firedrone.client as fdc
from firedrone.client.errors import FireDroneClientHttpError

In [3]:
import numpy as np
from IPython.display import Image
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Conv2D, Flatten
from keras import layers, models, optimizers

Using TensorFlow backend.


### Import CNN model

In [4]:
model = load_model('model_valacc_87_JL.h5')

W0806 00:09:04.079647 15064 deprecation_wrapper.py:119] From c:\users\john light\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 00:09:04.097628 15064 deprecation_wrapper.py:119] From c:\users\john light\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 00:09:04.112557 15064 deprecation_wrapper.py:119] From c:\users\john light\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0806 00:09:04.159464 15064 deprecation_wrapper.py:119] From c:\users\john light\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:131: The name tf.get_default_graph is deprec

### Create workspace and start run

In [5]:
# Use any convenient way to get your FireDrone API Key.
# You should use the user token you received when registering to the contest.
api_key = 'qfc1rk6^nFODCqB_Kkh_oDzO6mmeZ8L_vc$CVFLkR_-4K5YEoKKLRF@n0O^E_cq0'
# Create a new client workspace.
# -------------------------------------------------------------------------------------------
workspace = fdc.Workspace(api_key)
# Get a list of available scenes.
# -------------------------------------------------------------------------------------------
scenes = workspace.get_scenes()
print(scenes)

Sending GET request to https://api.firedrone.ai/v1/scenes/all
[{'id': 20, 'name': 'No-fire scene', 'description': 'This is a really challenging scene.', 'isDisabled': False}, {'id': 21, 'name': 'Notre-Dame in flames', 'description': 'This is a really challenging scene.', 'isDisabled': False}, {'id': 22, 'name': 'Misleading Scene', 'description': 'This is a beautiful sunset (also real). Unfortunatelly, it looks a lot like fire in the sky.', 'isDisabled': False}, {'id': 23, 'name': 'Large Street Fire Scene', 'description': 'A large street fire in a city. The fire has a complex structure which blends with some smoke patterns.', 'isDisabled': False}, {'id': 24, 'name': 'City Fire with Smoke Scene', 'description': 'An area of a city engulfed in flames. One large fire and several smaller ones can be spotted.', 'isDisabled': False}]


In [13]:
# Start a new direct run
# Use exception handling just in case you have not terminated a previous run.
# If that is the case, the error message will instruct you to run the END operation on the active run to end it.
# -------------------------------------------------------------------------------------------

try:
    start_result = workspace.directrun_start(20)
    print(start_result)
except FireDroneClientHttpError as e:
    display(e)

Sending POST request to https://api.firedrone.ai/v1/directruns/start?sceneid=20
{'uniqueId': 'f13813db-e82b-4442-b05b-e25ac5fdc4f9'}


In [14]:
# Save the run id for later
# -------------------------------------------------------------------------------------------
run_id = start_result['uniqueId']
run_id

'f13813db-e82b-4442-b05b-e25ac5fdc4f9'

### Measure dimensions of scene

In [15]:
width = 0
height = 0
move_result = workspace.directrun_move_down(run_id)
move_result['success'] = True
while move_result['success'] == True:
    move_result = workspace.directrun_move_right(run_id)
    
move_result['success'] = True
while move_result['success'] == True:
    move_result = workspace.directrun_move_up(run_id)
    width  += 1 
    
move_result['success'] = True
while move_result['success'] == True:
    move_result = workspace.directrun_move_left(run_id)
    height  += 1 

Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/movedown
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending POS

### Traverse scene looking for fire

In [ ]:
def predictFrame():
    #request and save frame
    frame = workspace.get_drone_fov_image(run_id)
    with open('./frame.png', 'wb') as f:
       f.write(frame)
    #convert frame to array for model input
    img_array = []
    img = load_img("frame.png")
    img_array.append(img_to_array(img))
    img_array = np.array(img_array)
    #predict if fire and submit score
    Prediction = model.predict(img_array)
    if Prediction >= .5:
        score_result = workspace.directrun_score(run_id, True)
        print("fire")
    else:
        #score_result = workspace.directrun_score(run_id, False)
        print("nofire")
    print(Prediction)

#traverse through scene looking for fire
for x in range(width):
    for y in range(height):
        move_result = workspace.directrun_move_down(run_id)
        predictFrame()
    if width - 1 > x:
        move_result = workspace.directrun_move_right(run_id)
        predictFrame()
        x += 1
    else:
        break
    for z in range(height):
        move_result = workspace.directrun_move_up(run_id)
        predictFrame()
    if width - 1 > x:
        move_result = workspace.directrun_move_right(run_id)
        predictFrame()
        x += 1
    else:
        break

Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/movedown
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/movedown
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/movedown
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/movedown
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to 

nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveright
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST re

nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST reque

nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST reque

nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/moveup
Sending GET request to https://api.firedrone.ai/v1/directruns/f13813db-e82b-4442-b05b-e25ac5fdc4f9/dronefieldofviewimage
nofire
[[0.]]
Sending POST reque

### End the run and view history

In [ ]:
# End the run
workspace.directrun_end(run_id)

In [ ]:
run_history = workspace.directrun_get_history()
print(run_history)

In [ ]:
run_history_entry = workspace.directrun_get_history_entry('2019_08_02_01_41_13_cbfd8c28-8a33-40bd-b9d1-f7f0352b6333')
print(run_history_entry)